**CURSO**: *Machine Learning* en Geociencias<br />
**Profesor**: Edier Aristizábal (evaristizabalg@unal.edu.co) <br />
**Credits**: The content of this notebook is taken from several sources, such as: [Ashish Arora in Medium](https://medium.com/@ashisharora2204/logistic-regression-maximum-likelihood-estimation-gradient-descent-a7962a452332). Every effort has been made to trace copyright holders of the materials used in this notebook. The author apologies for any unintentional omissions and would be pleased to add an acknowledgment in future editions.


# 16: Regresión Logística

El modelo de RL corresponde a un modelo supervisado para clasificaciones.

In [1]:
import numpy as np
import pandas as pd
#import warnings
#warnings.simplefilter("ignore")

En este caso se van a utilizar los datos de la base de datos para un modelo de predicción de movimientos en masa, donde la variable objetivo es 1 o 0, que corresponde a si la celda corresponde a un movimiento en masa o no, respectivamente.

In [26]:
df= pd.read_excel('https://github.com/edieraristizabal/MachineLearning/raw/master/data/BD_lamiel.xlsx', sheet_name='Sheet1')
df.head(2)

,Unnamed: 0,inventario,pendiente,aspecto,geologia
0,0,0,10.862183,208.523560,14
1,1,0,12.265345,207.437332,14


Para formar la matriz *X* con las variables predictoras se debe eliminar la columna con el inventario de movimientos en masa, que corresponde a la variable dependiente *y*, y una columna, para este caso, denominada *Unnamed: 0* que representa los índices de las filas importados de Excel.

In [24]:
X=df.drop(['inventario','Unnamed: 0'],axis=1)
X.head(2)

,pendiente,aspecto,geologia
0,10.862183,208.523560,14
1,12.265345,207.437332,14


In [25]:
y=df['inventario']
y.head(2)

0    0
1    0
Name: inventario, dtype: int64

## Statsmodels

Al igual que en el caso de modelos lineales, una buena opción es la librería statsmodels, ya que arroja la hoja de resultados con diferentes métricas que permiten conocer los resultados directamente.

In [3]:
import statsmodels.api as sm

Como la variable geologia es categórica se debe transformar a variables binarias o dummies. Inicialmente es importante saber cuantas clases tiene:

In [31]:
print(np.unique(X['geologia']))

[ 2  4  6  8  9 10 11 14 15 16]


Y posteriormente se transnforma con la siguiente función:

In [33]:

dummy_geologia=pd.get_dummies(X['geologia'],prefix='geo')
dummy_geologia.head(2)

,geo_2,geo_4,geo_6,geo_8,geo_9,geo_10,geo_11,geo_14,geo_15,geo_16
0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,1,0,0


Luego se debe incorporar las variables binarias creadas y remover la variable previa que contenía la geología.

In [29]:
column_name=X.columns.values.tolist()
column_name.remove('geologia')
X1=X[column_name].join(dummy_geologia)
X1.head()

,pendiente,aspecto,geo_2,geo_4,geo_6,geo_8,geo_9,geo_10,geo_11,geo_14,geo_15,geo_16
0,10.862183,208.523560,0,0,0,0,0,0,0,1,0,0
1,12.265345,207.437332,0,0,0,0,0,0,0,1,0,0
2,12.469252,202.684647,0,0,0,0,0,0,0,1,0,0
3,13.148026,211.619766,0,0,0,0,0,0,0,1,0,0
4,14.091524,220.028976,0,0,0,0,0,0,0,1,0,0


Como las clases de las variables categóricas son transformadas en variables binarias, se forma multicolinealidad, ya que las variables binarias creadas conforman sumandolos un vector de valores 1. Para eliminar esta multicolinealidad se debe eliminar una variable, por lo que los coeficientes de las demas clases son con respecto a dicha clase, y el coeficiente de la clase eliminada se encuentra dentro del intercepto. 

En este caso eliminaremos la variable 'geo_2'.

In [35]:
X1.drop('geo_2',axis=1,inplace=True)
X1.head()

,pendiente,aspecto,geo_4,geo_6,geo_8,geo_9,geo_10,geo_11,geo_14,geo_15,geo_16
0,10.862183,208.523560,0,0,0,0,0,0,1,0,0
1,12.265345,207.437332,0,0,0,0,0,0,1,0,0
2,12.469252,202.684647,0,0,0,0,0,0,1,0,0
3,13.148026,211.619766,0,0,0,0,0,0,1,0,0
4,14.091524,220.028976,0,0,0,0,0,0,1,0,0


La función Logit de *statsmodels* no incluye por defecto el intercepto, por lo tanto se debe adicionar a las variables independientes una columna con una constante de tal forma que se pueda obtener el intercepto.

In [36]:
model1=sm.Logit(y,sm.add_constant(X1))
result1=model1.fit(method='nm')
print(result1.summary())

c:\Users\edier\miniconda3\envs\carto\Lib\site-packages\statsmodels\base\optimizer.py:736: RuntimeWarning: Maximum number of iterations has been exceeded.
  retvals = optimize.fmin(f, start_params, args=fargs, xtol=xtol,
c:\Users\edier\miniconda3\envs\carto\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                           Logit Regression Results                           
Dep. Variable:             inventario   No. Observations:               910801
Model:                          Logit   Df Residuals:                   910789
Method:                           MLE   Df Model:                           11
Date:                Mon, 27 Nov 2023   Pseudo R-squ.:                  -35.70
Time:                        19:15:51   Log-Likelihood:            -4.3586e+05
converged:                      False   LL-Null:                       -11876.
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0001      0.009     -0.016      0.987      -0.017       0.017
pendiente  -4.336e-05      0.000     -0.218      0.827      -0.000       0.000
aspecto       -0.0024   2.32e-05   -102.300      0.0

Entre los resultados se destacan el estimador para los coeficientes (maxima verosimilitud -MLE-), el logaritmo del estimador MLE (Log-likelihood), el coeficiente de ajuste (Pseudo R-squ.). Con respecto a los coeficientes se presenta el valor de la prueba de hipótesis nula que el valor del coeficiente sea igual a cero (z) como el valor del coeficiente y el error estandar (std err), y el *p-value* (P>|z|), este valor debe ser menor al 5% (0.05), lo cual significa que la probabilidad que el coeficiente tenga un valor de 0 es muy bajo.  Finalmente se presenta el rango del 95% del dominio del valor del coeficiente, si el coeficiente es estadísticamente significativo, dicho rango no debe contener el valor 0.

Para obtener los pesos de las variables o coeficientes, se puede utilizar también la siguiente función.

In [37]:
result1.params

const       -0.000144
pendiente   -0.000043
aspecto     -0.002373
geo_4        0.000563
geo_6       -0.000169
geo_8        0.000089
geo_9        0.000020
geo_10       0.000597
geo_11      -0.000045
geo_14       0.000722
geo_15       0.000561
geo_16       0.000023
dtype: float64

Existen diferentes métodos para resolver el problema de regresión, por lo tanto se puede modificar dicho argumento.

In [38]:
result1=model1.fit(method='bfgs')
print(result1.summary())

c:\Users\edier\miniconda3\envs\carto\Lib\site-packages\scipy\optimize\_optimize.py:1359: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)
c:\Users\edier\miniconda3\envs\carto\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         Current function value: 0.012931
         Iterations: 35
         Function evaluations: 49
         Gradient evaluations: 49
                           Logit Regression Results                           
Dep. Variable:             inventario   No. Observations:               910801
Model:                          Logit   Df Residuals:                   910789
Method:                           MLE   Df Model:                           11
Date:                Mon, 27 Nov 2023   Pseudo R-squ.:                0.008328
Time:                        19:16:53   Log-Likelihood:                -11777.
converged:                      False   LL-Null:                       -11876.
Covariance Type:            nonrobust   LLR p-value:                 2.108e-36
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.3694      0.097    -65.668      0.000      -6.560      -6.1

En el siguiente link se pueden consultar las diferentes argumentos y opciones de la función RL en statsmodels.

https://tedboy.github.io/statsmodels_doc/generated/generated/statsmodels.api.Logit.fit.html#statsmodels.api.Logit.fit

También es posible utilizar la librería *statsmodels.formula.api*, la cual permite utilizar la nomenclatura *Platsy* que permite el uso de formulas, y mejora la comprensión e interpretación del modelo. Esta librearía incorpora por defecto el intercepto en el modelo y en las fórmulas pueden especificar el tipo de variables categóricas con una *C*. Esto permite que automáticamente el modelo convierta en binaria las clases de la variable categórica y elimine una de ellas, por defecto la primera. En caso de querer eliminar el intercepto se incorpora en la formula " - 1" 

In [18]:
import statsmodels.formula.api as sfm

In [20]:
lr   = sfm.logit(formula = "inventario ~ pendiente + C(geologia)", data = df).fit()
print(lr.summary())

         Current function value: 0.012661
         Iterations: 35


c:\Users\edier\miniconda3\envs\carto\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                           Logit Regression Results                           
Dep. Variable:             inventario   No. Observations:               910801
Model:                          Logit   Df Residuals:                   910790
Method:                           MLE   Df Model:                           10
Date:                Mon, 27 Nov 2023   Pseudo R-squ.:                 0.02902
Time:                        18:19:49   Log-Likelihood:                -11532.
converged:                      False   LL-Null:                       -11876.
Covariance Type:            nonrobust   LLR p-value:                1.220e-141
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -7.2365      0.089    -81.628      0.000      -7.410      -7.063
C(geologia)[T.4]     -8.0658     96.864     -0.083      0.934    -197.915     181.784
C(geologia)[T.6]    -12.

## Sklearn

La librería de Sklearn es mas ágil en resolver los problemas de RL. Como hiperparámetro utiliza el C, el cual regulariza los coeficientes, de forma similar a LASSO, pero de forma 1/C. Es por esto que no se obtienen coeficientes similares que para Statsmodels, se pueden aproximar asignando un valor para regularizar muy bajo o modificando el argumento *penalty=None*.

In [44]:
from sklearn.linear_model import LogisticRegression
#model = LogisticRegression(C=1e30)
model=LogisticRegression(penalty=None)

En el siguiente link se pueden conocer los diferentes argumentos de la función RL en Sklearn.

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

Luego de instanciar el modelo se ajusta a los datos y se pueden obtener tanto los coeficientes como el intercepto.

In [45]:
model2=model.fit(X1,y)
model2.coef_

c:\Users\edier\miniconda3\envs\carto\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([[ 3.98494503e-02,  1.39724415e-04, -3.52172475e-02,
        -5.08222818e-01,  3.91478101e-01, -2.74095331e+00,
        -3.88330356e-01,  2.31945099e-01, -5.55564633e-01,
         5.15636573e-01, -1.65907202e+00]])

In [46]:
model2.intercept_

array([-7.31819658])

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y)

A continuación se evaluán algunas de las métricas ya estudiadas en otros talleres.

In [65]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
Score_train=accuracy_score(y_train,model2.predict(X_train))
print('Precision en entrenamiento:',Score_train)
Score_test=accuracy_score(y_test,model2.predict(X_test))
print('Precision en validacion:',Score_test)
print(classification_report(y_test,model2.predict(X_test)))

Precision en entrenamiento: 1.0
Precision en validacion: 1.0


El método de RL puede arrojar valores categóricos en la predicción, en este caso 0 y 1, o probabilidades para cada caso, de la siguiente forma.

In [66]:
predictions = model2.predict(X_test)
probabilities = model2.predict_proba(X_test)[:, 1]
predictions,probabilities

El método de partición *cross validation* se puede utilizar de la siguiente manera:

In [64]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=5, random_state=1)
results = cross_val_score(model, X, y, cv=kfold)
print('Valor medio:',results.mean())
print('Desviacion estandar:',results.std())

Valor medio: 0.919047619047619
Desviacion estandar: 0.09786843869459148
